In [29]:
from pydantic_ai import Agent, RunContext
from pydantic_ai.models.openai import OpenAIModel
from pydantic import BaseModel, Field

from dataclasses import dataclass

import nest_asyncio
nest_asyncio.apply()

In [9]:
@dataclass
class User:
    name: str

In [25]:


open_aimodel = OpenAIModel(
    model_name='llama3.2',  
    base_url='http://localhost:11434/v1/', 
    api_key='ollama',
)

In [26]:
agent = Agent(
    open_aimodel,
    deps_type=User,
    result_type=bool,
    )


In [27]:
@agent.system_prompt
def add_user_name(ctx: RunContext[str]) -> str:
    return f"The user's name is {ctx.deps}"

In [28]:
result = agent.run_sync('Does their name start with "A"?', deps=User('Anne'))

In [14]:
print(result.data)

True


In [15]:
print(result.usage())

Usage(requests=2, request_tokens=372, response_tokens=34, total_tokens=406, details=None)


This example shows how to get structured responses from the agent.

Key concepts:
- Using Pydantic models we can define th ersponse template
- Type validation
- Field descriptions for better model understanding

In [30]:
class ResponseModel(BaseModel):
    """Structured response with metadata."""

    response: str
    needs_escalation: bool
    follow_up_required: bool
    sentiment: str = Field(description="Customer sentiment analysis")


In [31]:
agent2 = Agent(
    model=open_aimodel,
    result_type=ResponseModel,
    system_prompt=(
        "You are an intelligent customer support agent. "
        "Analyze queries carefully and provide structured responses."
    ),
)



In [32]:
response = agent2.run_sync("How can I track my order #12345?")
print(response.data.model_dump_json(indent=2))

{
  "response": "The status of your order number 12345 is currently pending shipping and will be delivered on the estimated date.",
  "needs_escalation": false,
  "follow_up_required": true,
  "sentiment": ""
}
